# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper`.

You will find more info about virtual environments [here](../../2.python/1.python_fundamentals/02.Package-Managers/00.Package_Managers.md).

Activate it and find a way to use it for this notebook.

## API Scraping

### A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword.

NOTE: external libraries need to be installed first. Check their documentation.

Check the [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the Requests documentation to:
1. use the `get()` method to connect to this endpoint: `https://country-leaders.herokuapp.com/status`
2. check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is the signification of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [52]:
# import the requests library (1 line)
import requests

# assign the root url (without /status) to the root_url variable for ease of reference (1 line)
root_url = "https://country-leaders.herokuapp.com"

# assign the /status endpoint to another variable called status_url (1 line)
status_url = "status"

# query the /status endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}/{status_url}")

# check the status_code using a condition and print appropriate messages (4 lines)
if req.status_code == 200:
    print(req.text)
else:
    print(req.status_code)


"Alive"


### Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. Check what to do in the Requests [Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/).


In [53]:
# Set the country_url variable (1 line)
country_url = "countries"

# query the /country endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(f"{root_url}/{country_url}")

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()

# display the request's status code and the countries variable (1 line)
print(req.status_code, countries)


403 {'message': 'The cookie is missing'}


### Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [54]:
# Set the cookie_url variable (1 line)
cookie_url = "cookie"

# Query the enpoint, set the cookies variable and display it (3 lines)
cookieReq = requests.get(f"{root_url}/{cookie_url}")
userCookie = cookieReq.cookies
print(userCookie)

<RequestsCookieJar[<Cookie user_cookie=0d533bae-d876-4343-a3ab-f912ed749392 for country-leaders.herokuapp.com/>]>


Try to query the countries endpoint using the cookie, save the output and print it.

In [55]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(f"{root_url}/{country_url}", cookies=userCookie)

# and check the status code and the countries variable's content (3 lines)
statCode = countries.status_code
print(statCode)
print(countries, countries.json())


200
<Response [200]> ['ru', 'fr', 'us', 'be', 'ma']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### Getting the actual data from the API

Query the `/leaders` endpoint.

In [56]:
# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(f"{root_url}/leaders")

# and check its content (4 lines)
if leaders == 200:
    print(leaders)
else:
    print(leaders, leaders.json())


<Response [403]> {'message': 'The cookie is missing'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.herokuapp.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [57]:
# Set the leaders_url variable (1 line)
leaders_url = "leaders"

# query the /leaders endpoint using cookies and parameters (take any country in countries)
# check the result (3 lines) 
leaders = requests.get(f"{root_url}/{leaders_url}?country=us", cookies=userCookie)   
result = leaders.json()
print(result)


[{'id': 'Q23', 'first_name': 'George', 'last_name': 'Washington', 'birth_date': '1732-02-22', 'death_date': '1799-12-14', 'place_of_birth': 'Westmoreland County', 'wikipedia_url': 'https://en.wikipedia.org/wiki/George_Washington', 'start_mandate': '1789-04-30', 'end_mandate': '1797-03-04'}, {'id': 'Q76', 'first_name': 'Barack', 'last_name': 'Obama', 'birth_date': '1961-08-04', 'death_date': None, 'place_of_birth': 'Kapiolani Medical Center for Women and Children', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Barack_Obama', 'start_mandate': '2009-01-20', 'end_mandate': '2017-01-20'}, {'id': 'Q91', 'first_name': 'Abraham', 'last_name': 'Lincoln', 'birth_date': '1809-02-12', 'death_date': '1865-04-15', 'place_of_birth': 'Sinking Spring Farm', 'wikipedia_url': 'https://en.wikipedia.org/wiki/Abraham_Lincoln', 'start_mandate': '1861-03-04', 'end_mandate': '1865-04-15'}, {'id': 'Q207', 'first_name': 'George', 'last_name': 'Bush', 'birth_date': '1946-07-06', 'death_date': None, 'place_of_bi

### A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each entry in a dictionary called `leaders_per_country` like this one: 
`{'fr':[{leader0}, {leader1},...],
'en':[{leader0}, {leader1},...],
...
}`.

In [58]:
# 5 lines
leaders_per_country = {}
for c in countries.json():
    res = requests.get(f"{root_url}/{leaders_url}?country={c}", cookies=userCookie)
    l = res.json()
    leaders_per_country[c] = l

It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [59]:
# 15 lines
def get_leaders():
    root_url = "https://country-leaders.herokuapp.com"
    status_url = "status"
    country_url = "countries"
    cookie_url = "cookie"
    leaders_url = "leaders"
    getCookie = requests.get(f"{root_url}/{cookie_url}")
    userCookie = getCookie.cookies
    getCountries = requests.get(f"{root_url}/{country_url}", cookies=userCookie)
    countries = getCountries.json()
    leaders_per_country = {}
    for c in getCountries.json():
        result = requests.get(f"{root_url}/{leaders_url}?country={c}", cookies=userCookie)
        l = result.json()
        leaders_per_country[c] = l
    return leaders_per_country

Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [60]:
# 2 lines
get_leaders()
print(leaders_per_country)

{'ru': [{'id': 'Q7747', 'first_name': 'Vladimir', 'last_name': 'Putin', 'birth_date': '1952-10-07', 'death_date': None, 'place_of_birth': 'Saint Petersburg', 'wikipedia_url': 'https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87', 'start_mandate': '2000-05-07', 'end_mandate': '2008-05-07'}, {'id': 'Q23530', 'first_name': 'Dmitry', 'last_name': 'Medvedev', 'birth_date': '1965-09-14', 'death_date': None, 'place_of_birth': 'Saint Petersburg', 'wikipedia_url': 'https://ru.wikipedia.org/wiki/%D0%9C%D0%B5%D0%B4%D0%B2%D0%B5%D0%B4%D0%B5%D0%B2,_%D0%94%D0%BC%D0%B8%D1%82%D1%80%D0%B8%D0%B9_%D0%90%D0%BD%D0%B0%D1%82%D0%BE%D0%BB%D1%8C%D0%B5%D0%B2%D0%B8%D1%87', 'start_mandate': '2008-05-07', 'end_mandate': '2012-05-07'}, {'id': 'Q34453', 'first_name': 'Boris', 'last_name': 'Yeltsin', 'birth_date': '1931-02-01', 'death_date': '2007-04-23', 'place_of_birth': 'Butka', 'wikipe

## Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [101]:
# 3 lines
import random
randCountry = random.choice(list(leaders_per_country))
randIdx = random.choice(range(0, len(leaders_per_country[randCountry])))
leader_url = leaders_per_country[randCountry][randIdx]['wikipedia_url']
query = requests.get(leader_url)
print(randCountry)
print(randIdx)
print(leader_url)


fr
11
https://fr.wikipedia.org/wiki/Albert_Lebrun


Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function to take a look. You will start the actual parsing in the next step.

In [102]:
# 3 lines
from bs4 import BeautifulSoup
soup = BeautifulSoup(query.content, 'html.parser')
soup.get_text()
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="fr">
 <head>
  <meta charset="utf-8"/>
  <title>
   Albert Lebrun — Wikipédia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":[",\t."," \t,"],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","janvier","février","mars","avril","mai","juin","juillet","août","septembre","octobre","novembre","décembre"],"wgRequestId":"50b7a586-c6df-40f2-8f00-13614e9769eb","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"Albert_Lebrun","wgTitle":"Albert Lebrun","wgCurRevisionId":196726091,"wgRevisionId":196726091,"wgArticleId":41737,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Article utilisant une Infobox","Page utilisant un sommaire limité","Article à référence nécessaire","Article avec une section vide o

That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [103]:
# 2 lines
paragraphs = soup.find_all('p')
for p in paragraphs:
    print(p)

<p>
		Pages pour les contributeurs déconnectés <a aria-label="En savoir plus sur la contribution" href="/wiki/Aide:Introduction"><span>en savoir plus</span></a>
</p>
<p class="sidebar-toc-title">
				Sommaire
				<button class="vector-toc-uncollapse-button">déplacer vers la barre latérale</button>
<button class="vector-toc-collapse-button">masquer</button>
</p>
<p>Pour les articles homonymes, voir <a class="mw-disambig" href="/wiki/Lebrun" title="Lebrun">Lebrun</a>.
</p>
<p class="mw-empty-elt">
</p>
<p><b>Albert Lebrun</b>, né le <time class="nowrap date-lien bday" data-sort-value="1871-08-29" datetime="1871-08-29"><a href="/wiki/29_ao%C3%BBt" title="29 août">29</a> <a href="/wiki/Ao%C3%BBt_1871" title="Août 1871">août</a> <a href="/wiki/1871" title="1871">1871</a></time> à <a href="/wiki/Mercy-le-Haut" title="Mercy-le-Haut">Mercy-le-Haut</a> (alors en <a href="/wiki/Moselle_(d%C3%A9partement)" title="Moselle (département)">Moselle</a>) et mort le <time class="nowrap date-lien dday" d

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one
  * store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
  * exit the loop

In [104]:
# 6 lines
for p in paragraphs:
    if p.find('b') == None:
        continue
    else:
        first_paragraph = p.get_text()
        print(first_paragraph)
        break

Albert Lebrun, né le 29 août 1871 à Mercy-le-Haut (alors en Moselle) et mort le 6 mars 1950 à Paris (alors dans le département de la Seine), est un homme d'État français. Il est président de la République française du 10 mai 1932 au 11 juillet 1940 (le dernier de la IIIe République).



At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [105]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
    query = requests.get(wikipedia_url)
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(query.content, 'html.parser')
    paragraphs = soup.find_all('p')
    for p in paragraphs:
        if p.find('b') == None:
            continue
        else:
            first_paragraph = p.get_text()
            break
    return first_paragraph

get_first_paragraph(leader_url)

https://fr.wikipedia.org/wiki/Albert_Lebrun


"Albert Lebrun, né le 29 août 1871 à Mercy-le-Haut (alors en Moselle) et mort le 6 mars 1950 à Paris (alors dans le département de la Seine), est un homme d'État français. Il est président de la République française du 10 mai 1932 au 11 juillet 1940 (le dernier de la IIIe\xa0République).\n"

### Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [106]:
# 4 lines
import re
compiled = re.compile("(\n)|(\[.*?\])|(Écouter)|(\(.*?uitspraak.*?\)\))|(\(listen\).*?;)|(/.*?/)")
sub_first_paragraph = re.sub(compiled, '', first_paragraph).strip()
print(sub_first_paragraph)

Albert Lebrun, né le 29 août 1871 à Mercy-le-Haut (alors en Moselle) et mort le 6 mars 1950 à Paris (alors dans le département de la Seine), est un homme d'État français. Il est président de la République française du 10 mai 1932 au 11 juillet 1940 (le dernier de la IIIe République).


Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [107]:
# 10 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
    query = requests.get(wikipedia_url)
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(query.content, 'html.parser')
    paragraphs = soup.find_all('p')
    for p in paragraphs:
        if p.find('b') == None:
            continue
        else:
            paragraph = p.get_text()
            break
    import re
    compiled = re.compile("(\n)|(\[.*?\])|(Écouter)|(\(.*?uitspraak.*?\)\))|(\(listen\).*?;)|(/.*?/)")
    first_paragraph = re.sub(compiled, '', paragraph).strip()
    return first_paragraph

get_first_paragraph(leader_url)

https://fr.wikipedia.org/wiki/Albert_Lebrun


"Albert Lebrun, né le 29 août 1871 à Mercy-le-Haut (alors en Moselle) et mort le 6 mars 1950 à Paris (alors dans le département de la Seine), est un homme d'État français. Il est président de la République française du 10 mai 1932 au 11 juillet 1940 (le dernier de la IIIe\xa0République)."

Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [108]:
# < 20 lines
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
    query = requests.get(wikipedia_url)
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(query.content, 'html.parser')
    paragraphs = soup.find_all('p')
    for p in paragraphs:
        if p.find('b') == None:
            continue
        else:
            paragraph = p.get_text()
            break
    import re
    compiled = re.compile("(\n)|(\[.*?\])|(Écouter)|(\(.*?uitspraak.*?\)\))|(\(listen\).*?;)|(/.*?/)")
    first_sub = re.sub(compiled, '', paragraph).strip()
    second_sub = re.sub("(\(\s*?\))", '', first_sub).strip()
    first_paragraph = re.sub("(\s{2,})", ' ', second_sub).strip()
    return first_paragraph

get_first_paragraph(leader_url)

https://fr.wikipedia.org/wiki/Albert_Lebrun


"Albert Lebrun, né le 29 août 1871 à Mercy-le-Haut (alors en Moselle) et mort le 6 mars 1950 à Paris (alors dans le département de la Seine), est un homme d'État français. Il est président de la République française du 10 mai 1932 au 11 juillet 1940 (le dernier de la IIIe\xa0République)."

### Caching
Good job! This function will be called many times. First because there are many leaders to loop through. Second because you are developing your code and are likely to re-run your loop multiple times. Third because the API might change and contain new leaders and you might want to update your data periodically.

For all these reasons, it can be benificial to *cache* the result of the `get_first_paragraph()` function so that if it called with the same parameter, you expect the same output and thus do not need to query Wikipedia another time.

You will use the [lru_cache](https://docs.python.org/3/library/functools.html#functools.lru_cache) decorator from the `functools` module. It is a tiny code change that will speed things up big time. You can use a `maxsize` of None. If you have questions about decorators, you will find more information [here](../../2.python/2.python_advanced/07.Decorator/decorators.ipynb)

In [140]:
# < 20 lines
import functools
@functools.lru_cache(maxsize=None)
def get_first_paragraph(wikipedia_url):
    print(wikipedia_url)
    query = requests.get(wikipedia_url)
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(query.content, 'html.parser')
    paragraphs = soup.find_all('p') #find all paragraphs
    for p in paragraphs: #find first paragraph using bold parameter
        if p.find('b') == None:
            continue
        else:
            paragraph = p.get_text()
            break
    import re
    compiled = re.compile("(\n)|(\[.*?\])|(Écouter)|(\(.*?uitspraak.*?\)\))|(\(listen\).*?;)|(/.*?/)")
    first_sub = re.sub(compiled, '', paragraph).strip()
    second_sub = re.sub("(\(\s*?\))", '', first_sub).strip()
    first_paragraph = re.sub("(\s{2,})", ' ', second_sub).strip()
    return first_paragraph


https://fr.wikipedia.org/wiki/Albert_Lebrun


"Albert Lebrun, né le 29 août 1871 à Mercy-le-Haut (alors en Moselle) et mort le 6 mars 1950 à Paris (alors dans le département de la Seine), est un homme d'État français. Il est président de la République française du 10 mai 1932 au 11 juillet 1940 (le dernier de la IIIe\xa0République)."

Let's check the speedup!

In [110]:
%%timeit
# Just run the cell. 
get_first_paragraph(leader_url)

https://fr.wikipedia.org/wiki/Albert_Lebrun
The slowest run took 16.00 times longer than the fastest. This could mean that an intermediate result is being cached.
686 ns ± 1.03 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [164]:
# 19 lines
def get_leaders():
    root_url = "https://country-leaders.herokuapp.com"
    country_url = "countries"
    cookie_url = "cookie"
    leaders_url = "leaders"
    getCookie = requests.get(f"{root_url}/{cookie_url}")
    userCookie = getCookie.cookies
    getCountries = requests.get(f"{root_url}/{country_url}", cookies=userCookie)
    countries = getCountries.json()
    leaders_per_country = {}
    for c in countries:
        result = requests.get(f"{root_url}/{leaders_url}?country={c}", cookies=userCookie)
        l = result.json()
        leaders_per_country[c] = l
    for country in leaders_per_country:
        leaders_list = leaders_per_country[country]
        for leader in leaders_list:
            leader_wikipedia_url = leader['wikipedia_url']
            first_paragraph = get_first_paragraph(leader_wikipedia_url)
            leader['first_paragraph'] = first_paragraph
        new_leaders_list = leaders_list
        leaders_per_country[country] = new_leaders_list
    return leaders_per_country
            

In [167]:
# Check the output of your function (2 lines)
output = get_leaders()
print(output)

{'ru': [{'id': 'Q7747', 'first_name': 'Vladimir', 'last_name': 'Putin', 'birth_date': '1952-10-07', 'death_date': None, 'place_of_birth': 'Saint Petersburg', 'wikipedia_url': 'https://ru.wikipedia.org/wiki/%D0%9F%D1%83%D1%82%D0%B8%D0%BD,_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80_%D0%92%D0%BB%D0%B0%D0%B4%D0%B8%D0%BC%D0%B8%D1%80%D0%BE%D0%B2%D0%B8%D1%87', 'start_mandate': '2000-05-07', 'end_mandate': '2008-05-07', 'first_paragraph': 'Влади́мир Влади́мирович Пу́тин (род. 7 октября 1952, Ленинград, СССР)\xa0— российский государственный, политический и военный деятель. Действующий президент Российской Федерации, председатель Государственного Совета Российской Федерации и верховный главнокомандующий Вооружёнными силами Российской Федерации с 7\xa0мая 2012 года. Ранее занимал должность президента с 7\xa0мая 2000\xa0года по 7\xa0мая 2008 года, также в 1999—2000 и 2008—2012 годах занимал должность председателя правительства Российской Федерации. Кандидат экономических наук (1997).'}, {'id

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function to check if the `status_code` is a cookie error. In which case, get new ones and query the api again.

If not, you may have a fast connection or a bit of luck. Do it anyway.

In [ ]:
# 22 lines


Check the output of your function again.

In [ ]:
%%time
# Check the output of your function (1 line)


Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [ ]:
# <20 lines

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
# 22 lines


Test your new functions.



In [ ]:
# Just run it.
%%time
leaders_per_country = get_leaders()

Using a Session makes the first call to `get_leaders()` faster however, for technical reasons, the `@lru_cache` decorator no longer works. 

Problem: The `session` parameter is not *hashable* and since the cache is a dictionary using all the parameters as key, this poses a problem.

Solution: Create our own cache that only cares about the url.

The code for this is provided. You should only read it.

In [ ]:
cache = {}
def hashable_cache(f):
    def inner(url, session):
        if url not in cache:
            cache[url] = f(url, session)
        return cache[url]
    return inner

Overwrite the `get_first_paragraph()` function to use the new decorator like this: `@hashable_cache`.


In [ ]:
# <20 lines

Check that the cache is working again.

In [ ]:
# Just run it.
%%time
leaders_per_country = get_leaders()

## Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Make a tour in our [file handling section](../../2.python/2.python_advanced/04.File-handling/file_handling.ipynb) to know more about how to read and write files using Python.

In [ ]:
# 3 lines


Make sure the file can be read back. Write the code to read the file.

In [ ]:
#3 lines

Make a function `save()` to call this code easily.

In [ ]:
# 3 lines


In [ ]:
# Call the function (1 line)

## Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## To go further

If you want to practice scraping, you can read this section and tackle the exercise.

You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!